# 1. Выборка данных

In [1]:
# Импортируем необходимые библиотеки
from sklearn.preprocessing import OrdinalEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression
from datetime import datetime, date, time
from requests.utils import quote
import requests
import pandas as pd
import numpy as np
import xlrd
import asyncio
import aiohttp
from time import sleep
from statistics import mean

Нам необходимо идентифицировать релевантные файлы данные из набора данных и определить, какие столбцы в наших данных подойдут для анализа.

In [4]:
datasets_list = []

for i in range(25):
    if (i < 13):
        exec("df" + str(i) + " = pd.read_excel('.../datasets/atp_20" + (("0" + str(i))[-2:]) + ".xls')")
    else:
        exec("df" + str(i) + " = pd.read_excel('.../datasets/atp_20" + (("0" + str(i))[-2:]) + ".xlsx')")
        
    exec("datasets_list.append(df" + str(i) + ")")

B:\Python\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
B:\Python\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
B:\Python\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)
B:\Python\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Unknown extension is not supported and will be removed
  warn(msg)


In [5]:
df0.loc[:, 'ATP':'Winner'].head(2)

,ATP,Location,Tournament,Date,Series,Court,Surface,Round,Best of,Winner
0,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Dosedel S.
1,1,Adelaide,Australian Hardcourt Championships,2000-01-03,International,Outdoor,Hard,1st Round,3,Enqvist T.


In [6]:
df0.loc[:, 'Winner':'Comment'].head(2)

,Winner,Loser,WRank,LRank,W1,L1,W2,L2,W3,L3,W4,L4,W5,L5,Wsets,Lsets,Comment
0,Dosedel S.,Ljubicic I.,63,77,6.0,4.0,6.0,2.0,NaN,NaN,NaN,NaN,NaN,NaN,2,0,Completed
1,Enqvist T.,Clement A.,5,56,6.0,3.0,6.0,3.0,NaN,NaN,NaN,NaN,NaN,NaN,2,0,Completed


In [6]:
players_df = pd.read_csv('.../datasets/atp_players.csv')

# 2. Очистка данных

In [7]:
# Удаляем WPts и LPts, так как это приведёт к утечке данных
for i in range(5, 25):
    exec("col = df" + str(i) + ".pop('WPts')")
    exec("df" + str(i) + ".insert(" + "df" + str(i) + ".shape[1], 'WPts', col)")
    exec("col = df" + str(i) + ".pop('LPts')")
    exec("df" + str(i) + ".insert(" + "df" + str(i) + ".shape[1], 'LPts', col)")

In [8]:
# Вертикально объединяем наборы данных
merged_df = pd.concat(datasets_list, ignore_index=True)

merged_df = merged_df.loc[:, 'ATP':'Comment'].query('Comment == "Completed" and W2.notnull() and LRank.notnull() and WRank.notnull() and LRank != "NR"')

merged_df.drop(merged_df.columns[[0, 2, 7, 25]], axis=1, inplace=True)

In [9]:
# Рассчитываем процент сетов, выигранных победителем матча
merged_df["Wsets_percent"] = merged_df["Wsets"]/(merged_df["Wsets"] + merged_df["Lsets"])

# Заменяем пробелы на нули в столбцах счета третьего сета
merged_df['W3'] = merged_df['W3'].replace(' ', 0)
merged_df['L3'] = merged_df['L3'].replace(' ', 0)

# Заполняем все оставшиеся пропущенные значения нулями
merged_df = merged_df.fillna(0)

# Рассчитываем среднее количество очков за сет для победителя и проигравшего
merged_df['W_round_pts_avg'] = (merged_df["W1"] + merged_df["W2"] + merged_df["W3"] + merged_df["W4"] + merged_df["W5"])/(merged_df["Wsets"] + merged_df["Lsets"])
merged_df['L_round_pts_avg'] = (merged_df["L1"] + merged_df["L2"] + merged_df["L3"] + merged_df["L4"] + merged_df["L5"])/(merged_df["Wsets"] + merged_df["Lsets"])

# Рассчитываем долю очков победителя от общего количества
merged_df['W_round_pts_avg_percent'] = merged_df['W_round_pts_avg']/(merged_df['W_round_pts_avg'] + merged_df['L_round_pts_avg'])

# Создаем нормализованный признак разницы рейтингов игроков
merged_df['W_over_L_rank_difference_normalized'] = np.log(merged_df['LRank']/merged_df['WRank'])

# Удаляем ненужные столбцы для упрощения датасета и уменьшения шума
merged_df.drop(merged_df.columns[[5, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 23, 24]], axis=1, inplace=True)

C:\Users\markz\AppData\Local\Temp\ipykernel_11804\3845507832.py:5: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df['W3'] = merged_df['W3'].replace(' ', 0)
C:\Users\markz\AppData\Local\Temp\ipykernel_11804\3845507832.py:6: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  merged_df['L3'] = merged_df['L3'].replace(' ', 0)
C:\Users\markz\AppData\Local\Temp\ipykernel_11804\3845507832.py:9: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_

In [10]:
# Создаем датафрейм с основными данными для анализа
dates_df = merged_df[['Location', 'Winner', 'Loser', 'Date']].copy()

# Преобразуем данные в длинный формат для объединения победителей и проигравших
dates_df = dates_df.melt(id_vars=['Location', 'Date'], value_vars=['Winner', 'Loser'], var_name='Player Type', value_name='Player')

# Определяем первую дату появления каждого игрока в каждом городе
dates_df = dates_df.groupby(['Location', 'Player']).agg({'Date': 'min'}).reset_index()
dates_df = dates_df.rename(columns={'Date': 'Player_min_date'})

# Присоединяем информацию о первой игре для победителя
merged_df = pd.merge(merged_df, dates_df, left_on=['Location', 'Winner'], right_on=['Location', 'Player'], how='left')

# Присоединяем информацию о первой игре для проигравшего
merged_df = pd.merge(merged_df, dates_df, left_on=['Location', 'Loser'], right_on=['Location', 'Player'], how='left')

# Создаем признак: играл ли победитель ранее в этом месте
merged_df['W_played_here_before'] = merged_df['Date'] > merged_df['Player_min_date_x']

# Создаем признак: играл ли проигравший ранее в этом месте
merged_df['L_played_here_before'] = merged_df['Date'] > merged_df['Player_min_date_y']

# Удаляем временные столбцы
merged_df = merged_df.drop(columns=['Player_min_date_x', 'Player_min_date_y', 'Player_x', 'Player_y'])
merged_df

,Location,Date,Series,Court,Surface,Winner,Loser,Wsets_percent,W_round_pts_avg_percent,W_over_L_rank_difference_normalized,W_played_here_before,L_played_here_before
0,Adelaide,2000-01-03,International,Outdoor,Hard,Dosedel S.,Ljubicic I.,1.000000,0.666667,0.200671,False,False
1,Adelaide,2000-01-03,International,Outdoor,Hard,Enqvist T.,Clement A.,1.000000,0.666667,2.415914,False,False
2,Adelaide,2000-01-03,International,Outdoor,Hard,Escude N.,Baccanello P.,0.666667,0.558824,2.795756,False,False
3,Adelaide,2000-01-03,International,Outdoor,Hard,Federer R.,Knippschild J.,1.000000,0.705882,0.291521,False,False
4,Adelaide,2000-01-03,International,Outdoor,Hard,Fromberg R.,Woodbridge T.,0.666667,0.514286,0.893818,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...
64135,Turin,2024-11-15,Masters Cup,Indoor,Hard,Zverev A.,Alcaraz C.,1.000000,0.565217,0.405465,True,True
64136,Turin,2024-11-15,Masters Cup,Indoor,Hard,Ruud C.,Rublev A.,0.666667,0.566667,0.133531,True,True
64137,Turin,2024-11-16,Masters Cup,Indoor,Hard,Fritz T.,Zverev A.,0.666667,0.516129,-0.916291,True,True
64138,Turin,2024-11-16,Masters Cup,Indoor,Hard,Sinner J.,Ruud C.,1.000000,0.800000,1.945910,True,True


Работа с данными из API.

In [11]:
# Нам нужно получить широту и долготу для каждого уникального города
weather_dataset = merged_df.query('Court == "Outdoor"').loc[:, : 'Date'].drop_duplicates()
notweather_dataset = merged_df.query('Court == "Indoor"').loc[:, : 'Date'].drop_duplicates()

locations = list(merged_df['Location'].drop_duplicates().reset_index(drop=True))

# Создаем список для хранения координат
latitudes = []
longitudes = []

# Настройка User-Agent для запросов к Nominatim
headers = {
    'User-Agent': 'TennisDataAnalysis/1.0'
}

for location in locations:
        encoded_location = quote(location, encoding='utf-8')
        url = f"https://nominatim.openstreetmap.org/search?q={location}&format=json"
        response = requests.get(url, headers=headers)
        data = response.json()
        lat = data[0]["lat"]
        lon = data[0]["lon"]
        latitudes.append(lat)
        longitudes.append(lon)

# Создаем датасет с координатами
data_tuples = list(zip(locations, latitudes, longitudes))
locations_dataset = pd.DataFrame(data_tuples, columns=['Location', 'Latitude', 'Longitude'])

# Фильтруем только те локации, для которых нашлись координаты
locations_dataset = locations_dataset.dropna(subset=['Latitude', 'Longitude'])

# Объединяем с погодным датасетом
weather_dataset = pd.merge(weather_dataset, locations_dataset, on='Location', how='inner')

weather_dataset

,Location,Date,Latitude,Longitude
0,Adelaide,2000-01-03,-34.9281805,138.5999312
1,Chennai,2000-01-03,13.0836939,80.2701860
2,Doha,2000-01-03,25.2856329,51.5264162
3,Auckland,2000-01-10,-36.8520950,174.7631803
4,Sydney,2000-01-10,-33.8698439,151.2082848
...,...,...,...,...
8158,Shanghai,2024-10-09,31.2312707,121.4700152
8159,Shanghai,2024-10-10,31.2312707,121.4700152
8160,Shanghai,2024-10-11,31.2312707,121.4700152
8161,Shanghai,2024-10-12,31.2312707,121.4700152


In [12]:
# Функци для получения погодных данных по долготе и ширине в конкретный день
async def get_weather_info(session, latitude, longitude, game_date):
    API = f'https://archive-api.open-meteo.com/v1/archive?latitude={latitude}&longitude={longitude}&start_date={game_date}&end_date={game_date}&hourly=temperature_2m,windspeed_10m,relativehumidity_2m,surface_pressure,precipitation,cloudcover,shortwave_radiation'
    async with session.get(API) as response:
        return await response.json()

latitude = list(weather_dataset['Latitude'])
longitude = list(weather_dataset['Longitude'])
dates = list(weather_dataset['Date'])
weather_info = []

formatted_dates = []
for date in weather_dataset['Date']:
    timestamp = pd.Timestamp(date)
    formatted_dates.append(timestamp.strftime('%Y-%m-%d'))

async with aiohttp.ClientSession() as session:
    tasks = []
    total_records = len(weather_dataset)
    batch_size = 10
    
    for i in range(total_records):
        # Создаем задачу
        task = asyncio.create_task(
            get_weather_info(
                session,
                weather_dataset['Latitude'].iloc[i],
                weather_dataset['Longitude'].iloc[i],
                formatted_dates[i]
            )
        )
        tasks.append(task)
        
        # Обрабатываем батч, когда достигаем размера batch_size
        if len(tasks) >= batch_size or i == total_records - 1:
            # Выполняем текущие задачи
            results = await asyncio.gather(*tasks, return_exceptions=True)
            
            # Обрабатываем результаты
            for idx, result in enumerate(results):
                max_retries = 5  # Максимальное количество попыток
                retry_count = 0
                current_result = result
                
                # Повторяем запрос, пока не получим корректный ответ или не исчерпаем попытки
                while not (isinstance(current_result, dict) and 'hourly' in current_result) and retry_count < max_retries:
                    print(f"Повторная попытка {retry_count+1}/{max_retries} для записи {i - len(tasks) + idx + 1}")
                    
                    # Создаем и выполняем новый запрос
                    new_task = asyncio.create_task(
                        get_weather_info(
                            session,
                            weather_dataset['Latitude'].iloc[i - len(tasks) + idx],
                            weather_dataset['Longitude'].iloc[i - len(tasks) + idx],
                            formatted_dates[i - len(tasks) + idx]
                        )
                    )
                    current_result = await new_task
                    retry_count += 1
                    
                    # Задержка перед повторной попыткой
                    await asyncio.sleep(1.0)
                
                # Добавляем результат в итоговый список
                if isinstance(current_result, dict) and 'hourly' in current_result:
                    weather_info.append(current_result)
                    print("good")
                else:
                    weather_info.append(None)
                    print(f"Не удалось получить данные после {max_retries} попыток")
  
            tasks = []
            
            await asyncio.sleep(1.0)

Повторная попытка 1/5 для записи 0
good
good
Повторная попытка 1/5 для записи 2
good
good
Повторная попытка 1/5 для записи 4
good
good
Повторная попытка 1/5 для записи 6
good
good
good
Повторная попытка 1/5 для записи 9
good
good
good
Повторная попытка 1/5 для записи 12
good
Повторная попытка 1/5 для записи 13
good
good
good
good
Повторная попытка 1/5 для записи 17
good
Повторная попытка 1/5 для записи 18
good
Повторная попытка 1/5 для записи 19
good
good
good
good
good
good
Повторная попытка 1/5 для записи 25
good
Повторная попытка 1/5 для записи 26
good
Повторная попытка 1/5 для записи 27
good
Повторная попытка 1/5 для записи 28
good
Повторная попытка 1/5 для записи 29
good
good
good
good
good
good
Повторная попытка 1/5 для записи 35
good
Повторная попытка 1/5 для записи 36
good
Повторная попытка 1/5 для записи 37
good
Повторная попытка 1/5 для записи 38
good
Повторная попытка 1/5 для записи 39
good
good
good
good
good
good
Повторная попытка 1/5 для записи 45
good
Повторная попытка 1

Теперь мы можем полноценно заполнить погодный датасет.

In [14]:
# Создаем списки для хранения извлеченных погодных данных
temperature = []
windspeed_10m = []
relativehumidity_2m = []
surface_pressure = []
precipitation = []
cloudcover = []
shortwave_radiation = []

# Извлекаем средние погодные показатели за дневные часы (11:00-17:00)
for i in range(len(weather_info)):
    temperature.append(mean(weather_info[i]['hourly']['temperature_2m'][11:17]))
    windspeed_10m.append(mean(weather_info[i]['hourly']['windspeed_10m'][11:17]))    
    relativehumidity_2m.append(mean(weather_info[i]['hourly']['relativehumidity_2m'][11:17]))
    surface_pressure.append(mean(weather_info[i]['hourly']['surface_pressure'][11:17]))
    precipitation.append(mean(weather_info[i]['hourly']['precipitation'][11:17]))
    cloudcover.append(mean(weather_info[i]['hourly']['cloudcover'][11:17]))
    shortwave_radiation.append(mean(weather_info[i]['hourly']['shortwave_radiation'][11:17]))

In [15]:
# Удаляем ненужные столбцы из погодного датасета(Latitude и Longitude)
weather_dataset.drop(weather_dataset.columns[[2,3]], axis=1, inplace=True)

# Добавляем новые стобцы в погодный датасет
weather_dataset['Temperature'] = temperature
weather_dataset['Windspeed_10m'] = windspeed_10m
weather_dataset['Relativehumidity_2m'] = relativehumidity_2m
weather_dataset['Surface_pressure'] = surface_pressure
weather_dataset['Precipitation'] = precipitation
weather_dataset['Сloudcover'] =cloudcover
weather_dataset['Shortwave_radiation'] = shortwave_radiation
weather_dataset

,Location,Date,Temperature,Windspeed_10m,Relativehumidity_2m,Surface_pressure,Precipitation,Сloudcover,Shortwave_radiation
0,Adelaide,2000-01-03,21.433333,9.650000,85.000000,1017.633333,0.0,72.000000,0.000000
1,Chennai,2000-01-03,24.016667,17.466667,69.666667,1010.133333,0.0,31.166667,73.166667
2,Doha,2000-01-03,24.016667,17.466667,69.666667,1010.133333,0.0,31.166667,73.166667
3,Auckland,2000-01-10,16.433333,20.450000,75.666667,1013.450000,0.0,38.500000,0.000000
4,Sydney,2000-01-10,16.433333,20.450000,75.666667,1013.450000,0.0,38.500000,0.000000
...,...,...,...,...,...,...,...,...,...
8158,Shanghai,2024-10-09,16.266667,10.900000,81.500000,1019.633333,0.0,41.000000,0.000000
8159,Shanghai,2024-10-10,15.416667,5.766667,84.166667,1016.300000,0.0,1.333333,0.000000
8160,Shanghai,2024-10-11,17.916667,8.600000,89.333333,1019.816667,0.0,58.666667,0.000000
8161,Shanghai,2024-10-12,20.566667,8.850000,84.833333,1020.450000,0.0,61.666667,0.000000


In [16]:
# Создаем искусственные данные(хорошие в среднем) для матчей в закрытых помещениях
notweather_dataset = notweather_dataset.reset_index(drop=True)
temperature = [23 for i in range(2504)]
shortwave_radiation_precipitation_windspeed_10m = [0 for i in range(2504)]
relativehumidity = [50 for i in range(2504)]
surface_pressure = [1013.25 for i in range(2504)]
сloudcover = [100 for i in range(2504)]

notweather_dataset['Temperature'] = temperature
notweather_dataset['Windspeed_10m'] = shortwave_radiation_precipitation_windspeed_10m
notweather_dataset['Relativehumidity_2m'] = relativehumidity
notweather_dataset['Surface_pressure'] = surface_pressure
notweather_dataset['Precipitation'] = shortwave_radiation_precipitation_windspeed_10m
notweather_dataset['Сloudcover'] = сloudcover
notweather_dataset['Shortwave_radiation'] = shortwave_radiation_precipitation_windspeed_10m
notweather_dataset

,Location,Date,Temperature,Windspeed_10m,Relativehumidity_2m,Surface_pressure,Precipitation,Сloudcover,Shortwave_radiation
0,Marseille,2000-02-07,23,0,50,1013.25,0,100,0
1,San Jose,2000-02-07,23,0,50,1013.25,0,100,0
2,Memphis,2000-02-14,23,0,50,1013.25,0,100,0
3,Rotterdam,2000-02-14,23,0,50,1013.25,0,100,0
4,London,2000-02-21,23,0,50,1013.25,0,100,0
...,...,...,...,...,...,...,...,...,...
2499,Turin,2024-11-13,23,0,50,1013.25,0,100,0
2500,Turin,2024-11-14,23,0,50,1013.25,0,100,0
2501,Turin,2024-11-15,23,0,50,1013.25,0,100,0
2502,Turin,2024-11-16,23,0,50,1013.25,0,100,0


In [17]:
full_weather_dataset = pd.concat([weather_dataset, notweather_dataset], ignore_index=True)
full_weather_dataset

,Location,Date,Temperature,Windspeed_10m,Relativehumidity_2m,Surface_pressure,Precipitation,Сloudcover,Shortwave_radiation
0,Adelaide,2000-01-03,21.433333,9.650000,85.000000,1017.633333,0.0,72.000000,0.000000
1,Chennai,2000-01-03,24.016667,17.466667,69.666667,1010.133333,0.0,31.166667,73.166667
2,Doha,2000-01-03,24.016667,17.466667,69.666667,1010.133333,0.0,31.166667,73.166667
3,Auckland,2000-01-10,16.433333,20.450000,75.666667,1013.450000,0.0,38.500000,0.000000
4,Sydney,2000-01-10,16.433333,20.450000,75.666667,1013.450000,0.0,38.500000,0.000000
...,...,...,...,...,...,...,...,...,...
10662,Turin,2024-11-13,23.000000,0.000000,50.000000,1013.250000,0.0,100.000000,0.000000
10663,Turin,2024-11-14,23.000000,0.000000,50.000000,1013.250000,0.0,100.000000,0.000000
10664,Turin,2024-11-15,23.000000,0.000000,50.000000,1013.250000,0.0,100.000000,0.000000
10665,Turin,2024-11-16,23.000000,0.000000,50.000000,1013.250000,0.0,100.000000,0.000000


In [18]:
full_weather_dataset.to_excel('.../datasets/full_weather_dataset.xlsx', index=False)

In [19]:
full_weather_dataset = pd.read_excel('.../datasets/full_weather_dataset.xlsx')
full_weather_dataset

,Location,Date,Temperature,Windspeed_10m,Relativehumidity_2m,Surface_pressure,Precipitation,Сloudcover,Shortwave_radiation
0,Adelaide,2000-01-03,21.433333,9.650000,85.000000,1017.633333,0.0,72.000000,0.000000
1,Chennai,2000-01-03,24.016667,17.466667,69.666667,1010.133333,0.0,31.166667,73.166667
2,Doha,2000-01-03,24.016667,17.466667,69.666667,1010.133333,0.0,31.166667,73.166667
3,Auckland,2000-01-10,16.433333,20.450000,75.666667,1013.450000,0.0,38.500000,0.000000
4,Sydney,2000-01-10,16.433333,20.450000,75.666667,1013.450000,0.0,38.500000,0.000000
...,...,...,...,...,...,...,...,...,...
10662,Turin,2024-11-13,23.000000,0.000000,50.000000,1013.250000,0.0,100.000000,0.000000
10663,Turin,2024-11-14,23.000000,0.000000,50.000000,1013.250000,0.0,100.000000,0.000000
10664,Turin,2024-11-15,23.000000,0.000000,50.000000,1013.250000,0.0,100.000000,0.000000
10665,Turin,2024-11-16,23.000000,0.000000,50.000000,1013.250000,0.0,100.000000,0.000000


In [20]:
merged_df = pd.merge(merged_df, full_weather_dataset, on=['Location', 'Date'])
merged_df.drop(merged_df.columns[[0]], axis=1, inplace=True)
merged_df

,Date,Series,Court,Surface,Winner,Loser,Wsets_percent,W_round_pts_avg_percent,W_over_L_rank_difference_normalized,W_played_here_before,L_played_here_before,Temperature,Windspeed_10m,Relativehumidity_2m,Surface_pressure,Precipitation,Сloudcover,Shortwave_radiation
0,2000-01-03,International,Outdoor,Hard,Dosedel S.,Ljubicic I.,1.000000,0.666667,0.200671,False,False,21.433333,9.65,85.0,1017.633333,0.0,72.0,0.0
1,2000-01-03,International,Outdoor,Hard,Enqvist T.,Clement A.,1.000000,0.666667,2.415914,False,False,21.433333,9.65,85.0,1017.633333,0.0,72.0,0.0
2,2000-01-03,International,Outdoor,Hard,Escude N.,Baccanello P.,0.666667,0.558824,2.795756,False,False,21.433333,9.65,85.0,1017.633333,0.0,72.0,0.0
3,2000-01-03,International,Outdoor,Hard,Federer R.,Knippschild J.,1.000000,0.705882,0.291521,False,False,21.433333,9.65,85.0,1017.633333,0.0,72.0,0.0
4,2000-01-03,International,Outdoor,Hard,Fromberg R.,Woodbridge T.,0.666667,0.514286,0.893818,False,False,21.433333,9.65,85.0,1017.633333,0.0,72.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
64135,2024-11-15,Masters Cup,Indoor,Hard,Zverev A.,Alcaraz C.,1.000000,0.565217,0.405465,True,True,23.000000,0.00,50.0,1013.250000,0.0,100.0,0.0
64136,2024-11-15,Masters Cup,Indoor,Hard,Ruud C.,Rublev A.,0.666667,0.566667,0.133531,True,True,23.000000,0.00,50.0,1013.250000,0.0,100.0,0.0
64137,2024-11-16,Masters Cup,Indoor,Hard,Fritz T.,Zverev A.,0.666667,0.516129,-0.916291,True,True,23.000000,0.00,50.0,1013.250000,0.0,100.0,0.0
64138,2024-11-16,Masters Cup,Indoor,Hard,Sinner J.,Ruud C.,1.000000,0.800000,1.945910,True,True,23.000000,0.00,50.0,1013.250000,0.0,100.0,0.0


Теперь можно начать работу с набором данных игроков.

In [21]:
# Оставляем только полезные для анализа колонки: имена, возраст, вес, рост и страну
players_df.drop(players_df.columns[[0, 1, 2, 3, 5, 7, 8, 9, 11, 12, 13, 14, 16, 17, 19, 20]], axis=1, inplace=True)

# Заполняем пропущенные значения медианой
players_df['birth_year'] = players_df['birth_year'].fillna(players_df['birth_year'].median())
players_df['weight_kg'] = players_df['weight_kg'].fillna(players_df['weight_kg'].median())
players_df['weight_kg'].replace(0, players_df['weight_kg'].median(), inplace=True)
players_df['height_cm'] = players_df['height_cm'].fillna(players_df['height_cm'].median())
players_df['height_cm'].replace(0, players_df['height_cm'].median(), inplace=True)
players_df['flag_code'] = players_df['flag_code'].fillna('UNK')
players_df['full_name'] = players_df['full_name'].apply(lambda x: x + '.')
players_df

C:\Users\markz\AppData\Local\Temp\ipykernel_11804\922669919.py:9: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  players_df['weight_kg'].replace(0, players_df['weight_kg'].median(), inplace=True)
C:\Users\markz\AppData\Local\Temp\ipykernel_11804\922669919.py:11: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values a

,full_name,flag_code,birth_year,weight_kg,height_cm
0,Acuna R.,CHI,1958.0,68.0,175.0
1,Abdullahi S.,NGR,1960.0,77.0,183.0
2,Aerts N.,BRA,1963.0,75.0,188.0
3,Adams E.,USA,1959.0,73.0,178.0
4,Agenor R.,USA,1964.0,82.0,180.0
...,...,...,...,...,...
10907,Zorab A.,UNK,1976.0,77.0,183.0
10908,Zimmermann J.,UNK,1976.0,77.0,183.0
10909,Zinn G.,UNK,1976.0,77.0,183.0
10910,Zenger C.,GER,1976.0,77.0,183.0


In [22]:
winner_players_df = players_df.copy()
winner_players_df = winner_players_df.rename(columns={'full_name': 'Winner', 'flag_code': 'W_flag_code', 'birth_year': 'W_birth_year', 'weight_kg': 'W_weight_kg', 'height_cm': 'W_height_cm'})

loser_players_df = players_df.copy()
loser_players_df = loser_players_df.rename(columns={'full_name': 'Loser', 'flag_code': 'L_flag_code', 'birth_year': 'L_birth_year', 'weight_kg': 'L_weight_kg', 'height_cm': 'L_height_cm'})

In [23]:
merged_df = pd.merge(merged_df, winner_players_df, on=['Winner'])
merged_df = pd.merge(merged_df, loser_players_df, on=['Loser'])
merged_df

,Date,Series,Court,Surface,Winner,Loser,Wsets_percent,W_round_pts_avg_percent,W_over_L_rank_difference_normalized,W_played_here_before,...,Сloudcover,Shortwave_radiation,W_flag_code,W_birth_year,W_weight_kg,W_height_cm,L_flag_code,L_birth_year,L_weight_kg,L_height_cm
0,2000-01-03,International,Outdoor,Hard,Dosedel S.,Ljubicic I.,1.000000,0.666667,0.200671,False,...,72.0,0.0,CZE,1970.0,79.0,183.0,CRO,1979.0,92.0,193.0
1,2000-01-03,International,Outdoor,Hard,Enqvist T.,Clement A.,1.000000,0.666667,2.415914,False,...,72.0,0.0,SWE,1974.0,88.0,191.0,FRA,1977.0,74.0,173.0
2,2000-01-03,International,Outdoor,Hard,Escude N.,Baccanello P.,0.666667,0.558824,2.795756,False,...,72.0,0.0,FRA,1976.0,70.0,185.0,AUS,1979.0,85.0,183.0
3,2000-01-03,International,Outdoor,Hard,Federer R.,Knippschild J.,1.000000,0.705882,0.291521,False,...,72.0,0.0,SUI,1981.0,85.0,185.0,GER,1975.0,82.0,191.0
4,2000-01-03,International,Outdoor,Hard,Fromberg R.,Woodbridge T.,0.666667,0.514286,0.893818,False,...,72.0,0.0,AUS,1970.0,88.0,196.0,AUS,1971.0,75.0,178.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57469,2024-11-13,Masters Cup,Indoor,Hard,Zverev A.,Ruud C.,1.000000,0.590909,1.252763,True,...,100.0,0.0,GER,1997.0,86.0,198.0,NOR,1972.0,75.0,188.0
57470,2024-11-13,Masters Cup,Indoor,Hard,Zverev A.,Ruud C.,1.000000,0.590909,1.252763,True,...,100.0,0.0,GER,1997.0,86.0,198.0,NOR,1998.0,77.0,183.0
57471,2024-11-15,Masters Cup,Indoor,Hard,Ruud C.,Rublev A.,0.666667,0.566667,0.133531,True,...,100.0,0.0,NOR,1972.0,75.0,188.0,RUS,1997.0,68.0,188.0
57472,2024-11-15,Masters Cup,Indoor,Hard,Ruud C.,Rublev A.,0.666667,0.566667,0.133531,True,...,100.0,0.0,NOR,1998.0,77.0,183.0,RUS,1997.0,68.0,188.0


In [24]:
# Сделаем функцию для расчета возраста для каждой строки
def calculate_age(row, player):
    if (player == "winner"):
        birth_year = row['W_birth_year']
    else:
        birth_year = row['L_birth_year']
    game_date = row['Date'].strftime('%Y-%m-%d')
    game_datetime = datetime.strptime(game_date, '%Y-%m-%d')
    player_age = game_datetime.year - birth_year
      
    return player_age


merged_df['W_age'] = merged_df.apply(lambda row: calculate_age(row, "winner"), axis=1)
merged_df['L_age'] = merged_df.apply(lambda row: calculate_age(row, "loser"), axis=1)
merged_df.drop(merged_df.columns[[19, 23]], axis=1, inplace=True)
merged_df

,Date,Series,Court,Surface,Winner,Loser,Wsets_percent,W_round_pts_avg_percent,W_over_L_rank_difference_normalized,W_played_here_before,...,Сloudcover,Shortwave_radiation,W_flag_code,W_weight_kg,W_height_cm,L_flag_code,L_weight_kg,L_height_cm,W_age,L_age
0,2000-01-03,International,Outdoor,Hard,Dosedel S.,Ljubicic I.,1.000000,0.666667,0.200671,False,...,72.0,0.0,CZE,79.0,183.0,CRO,92.0,193.0,30.0,21.0
1,2000-01-03,International,Outdoor,Hard,Enqvist T.,Clement A.,1.000000,0.666667,2.415914,False,...,72.0,0.0,SWE,88.0,191.0,FRA,74.0,173.0,26.0,23.0
2,2000-01-03,International,Outdoor,Hard,Escude N.,Baccanello P.,0.666667,0.558824,2.795756,False,...,72.0,0.0,FRA,70.0,185.0,AUS,85.0,183.0,24.0,21.0
3,2000-01-03,International,Outdoor,Hard,Federer R.,Knippschild J.,1.000000,0.705882,0.291521,False,...,72.0,0.0,SUI,85.0,185.0,GER,82.0,191.0,19.0,25.0
4,2000-01-03,International,Outdoor,Hard,Fromberg R.,Woodbridge T.,0.666667,0.514286,0.893818,False,...,72.0,0.0,AUS,88.0,196.0,AUS,75.0,178.0,30.0,29.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57469,2024-11-13,Masters Cup,Indoor,Hard,Zverev A.,Ruud C.,1.000000,0.590909,1.252763,True,...,100.0,0.0,GER,86.0,198.0,NOR,75.0,188.0,27.0,52.0
57470,2024-11-13,Masters Cup,Indoor,Hard,Zverev A.,Ruud C.,1.000000,0.590909,1.252763,True,...,100.0,0.0,GER,86.0,198.0,NOR,77.0,183.0,27.0,26.0
57471,2024-11-15,Masters Cup,Indoor,Hard,Ruud C.,Rublev A.,0.666667,0.566667,0.133531,True,...,100.0,0.0,NOR,75.0,188.0,RUS,68.0,188.0,52.0,27.0
57472,2024-11-15,Masters Cup,Indoor,Hard,Ruud C.,Rublev A.,0.666667,0.566667,0.133531,True,...,100.0,0.0,NOR,77.0,183.0,RUS,68.0,188.0,26.0,27.0


In [25]:
merged_df.to_excel('.../datasets/merged_df.xlsx', index=False)

# 3. Построение данных

In [26]:
merged_df = pd.read_excel('.../datasets/merged_df.xlsx')
merged_df

,Date,Series,Court,Surface,Winner,Loser,Wsets_percent,W_round_pts_avg_percent,W_over_L_rank_difference_normalized,W_played_here_before,...,Сloudcover,Shortwave_radiation,W_flag_code,W_weight_kg,W_height_cm,L_flag_code,L_weight_kg,L_height_cm,W_age,L_age
0,2000-01-03,International,Outdoor,Hard,Dosedel S.,Ljubicic I.,1.000000,0.666667,0.200671,False,...,72.0,0.0,CZE,79,183,CRO,92,193,30,21
1,2000-01-03,International,Outdoor,Hard,Enqvist T.,Clement A.,1.000000,0.666667,2.415914,False,...,72.0,0.0,SWE,88,191,FRA,74,173,26,23
2,2000-01-03,International,Outdoor,Hard,Escude N.,Baccanello P.,0.666667,0.558824,2.795756,False,...,72.0,0.0,FRA,70,185,AUS,85,183,24,21
3,2000-01-03,International,Outdoor,Hard,Federer R.,Knippschild J.,1.000000,0.705882,0.291521,False,...,72.0,0.0,SUI,85,185,GER,82,191,19,25
4,2000-01-03,International,Outdoor,Hard,Fromberg R.,Woodbridge T.,0.666667,0.514286,0.893818,False,...,72.0,0.0,AUS,88,196,AUS,75,178,30,29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57469,2024-11-13,Masters Cup,Indoor,Hard,Zverev A.,Ruud C.,1.000000,0.590909,1.252763,True,...,100.0,0.0,GER,86,198,NOR,75,188,27,52
57470,2024-11-13,Masters Cup,Indoor,Hard,Zverev A.,Ruud C.,1.000000,0.590909,1.252763,True,...,100.0,0.0,GER,86,198,NOR,77,183,27,26
57471,2024-11-15,Masters Cup,Indoor,Hard,Ruud C.,Rublev A.,0.666667,0.566667,0.133531,True,...,100.0,0.0,NOR,75,188,RUS,68,188,52,27
57472,2024-11-15,Masters Cup,Indoor,Hard,Ruud C.,Rublev A.,0.666667,0.566667,0.133531,True,...,100.0,0.0,NOR,77,183,RUS,68,188,26,27


In [28]:
# Инициализация OneHotEncoder
ohe_encoder = OneHotEncoder(sparse_output=False, handle_unknown='ignore') 

features = ["Series", "Court", "Surface"]

# Обучение и преобразование
ohe_encoder.fit(merged_df[features])
new_feats = ohe_encoder.transform(merged_df[features])

# Создание DataFrame с новыми признаками
new_cols = pd.DataFrame(new_feats, dtype=int, columns=ohe_encoder.get_feature_names_out(features))

# Объединение с исходным DataFrame
merged_df = pd.concat([merged_df, new_cols], axis=1)
merged_df.drop(features, axis=1, inplace=True)

merged_df

,Date,Winner,Loser,Wsets_percent,W_round_pts_avg_percent,W_over_L_rank_difference_normalized,W_played_here_before,L_played_here_before,Temperature,Windspeed_10m,...,Series_International Gold,Series_Masters,Series_Masters 1000,Series_Masters Cup,Court_Indoor,Court_Outdoor,Surface_Carpet,Surface_Clay,Surface_Grass,Surface_Hard
0,2000-01-03,Dosedel S.,Ljubicic I.,1.000000,0.666667,0.200671,False,False,21.433333,9.65,...,0,0,0,0,0,1,0,0,0,1
1,2000-01-03,Enqvist T.,Clement A.,1.000000,0.666667,2.415914,False,False,21.433333,9.65,...,0,0,0,0,0,1,0,0,0,1
2,2000-01-03,Escude N.,Baccanello P.,0.666667,0.558824,2.795756,False,False,21.433333,9.65,...,0,0,0,0,0,1,0,0,0,1
3,2000-01-03,Federer R.,Knippschild J.,1.000000,0.705882,0.291521,False,False,21.433333,9.65,...,0,0,0,0,0,1,0,0,0,1
4,2000-01-03,Fromberg R.,Woodbridge T.,0.666667,0.514286,0.893818,False,False,21.433333,9.65,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57469,2024-11-13,Zverev A.,Ruud C.,1.000000,0.590909,1.252763,True,True,23.000000,0.00,...,0,0,0,1,1,0,0,0,0,1
57470,2024-11-13,Zverev A.,Ruud C.,1.000000,0.590909,1.252763,True,True,23.000000,0.00,...,0,0,0,1,1,0,0,0,0,1
57471,2024-11-15,Ruud C.,Rublev A.,0.666667,0.566667,0.133531,True,True,23.000000,0.00,...,0,0,0,1,1,0,0,0,0,1
57472,2024-11-15,Ruud C.,Rublev A.,0.666667,0.566667,0.133531,True,True,23.000000,0.00,...,0,0,0,1,1,0,0,0,0,1


In [29]:
# Сделаем списки для кодировки
unique_flags = list(set(list(merged_df['W_flag_code'].unique()) + list(merged_df['L_flag_code'].unique())))
encoder_flags = [i for i in range(80)]
unique_players = list(set(list(merged_df['Winner'].unique()) + list(merged_df['Loser'].unique())))
encoder_players = [i for i in range(1178)]

# Созддим словари для кодирования столбцов «W_flag_code», «L_flag_code», «Победитель» и «Проигравший»
flag_dict = dict(zip(unique_flags, encoder_flags))
player_dict = dict(zip(unique_players, encoder_players))

# Используем словари кодировок для кодирования столбцов «W_flag_code», «L_flag_code», «Winner» и «Loser»
merged_df['W_flag_code'] = merged_df['W_flag_code'].map(flag_dict)
merged_df['L_flag_code'] = merged_df['L_flag_code'].map(flag_dict)
merged_df['Winner'] = merged_df['Winner'].map(player_dict)
merged_df['Loser'] = merged_df['Loser'].map(player_dict)

merged_df

,Date,Winner,Loser,Wsets_percent,W_round_pts_avg_percent,W_over_L_rank_difference_normalized,W_played_here_before,L_played_here_before,Temperature,Windspeed_10m,...,Series_International Gold,Series_Masters,Series_Masters 1000,Series_Masters Cup,Court_Indoor,Court_Outdoor,Surface_Carpet,Surface_Clay,Surface_Grass,Surface_Hard
0,2000-01-03,118.0,967.0,1.000000,0.666667,0.200671,False,False,21.433333,9.65,...,0,0,0,0,0,1,0,0,0,1
1,2000-01-03,606.0,937.0,1.000000,0.666667,2.415914,False,False,21.433333,9.65,...,0,0,0,0,0,1,0,0,0,1
2,2000-01-03,1110.0,61.0,0.666667,0.558824,2.795756,False,False,21.433333,9.65,...,0,0,0,0,0,1,0,0,0,1
3,2000-01-03,821.0,445.0,1.000000,0.705882,0.291521,False,False,21.433333,9.65,...,0,0,0,0,0,1,0,0,0,1
4,2000-01-03,973.0,1041.0,0.666667,0.514286,0.893818,False,False,21.433333,9.65,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57469,2024-11-13,630.0,1097.0,1.000000,0.590909,1.252763,True,True,23.000000,0.00,...,0,0,0,1,1,0,0,0,0,1
57470,2024-11-13,630.0,1097.0,1.000000,0.590909,1.252763,True,True,23.000000,0.00,...,0,0,0,1,1,0,0,0,0,1
57471,2024-11-15,1097.0,293.0,0.666667,0.566667,0.133531,True,True,23.000000,0.00,...,0,0,0,1,1,0,0,0,0,1
57472,2024-11-15,1097.0,293.0,0.666667,0.566667,0.133531,True,True,23.000000,0.00,...,0,0,0,1,1,0,0,0,0,1


# 4. Последняя стадия подготовки датасета

In [57]:
# Удаляем ещё столбцы, которые нам не нужны
df = merged_df.drop(["Wsets_percent", "W_round_pts_avg_percent"], axis = 1)
df

,Date,Winner,Loser,W_over_L_rank_difference_normalized,W_played_here_before,L_played_here_before,Temperature,Windspeed_10m,Relativehumidity_2m,Surface_pressure,...,Series_International Gold,Series_Masters,Series_Masters 1000,Series_Masters Cup,Court_Indoor,Court_Outdoor,Surface_Carpet,Surface_Clay,Surface_Grass,Surface_Hard
0,2000-01-03,118.0,967.0,0.200671,False,False,21.433333,9.65,85.0,1017.633333,...,0,0,0,0,0,1,0,0,0,1
1,2000-01-03,606.0,937.0,2.415914,False,False,21.433333,9.65,85.0,1017.633333,...,0,0,0,0,0,1,0,0,0,1
2,2000-01-03,1110.0,61.0,2.795756,False,False,21.433333,9.65,85.0,1017.633333,...,0,0,0,0,0,1,0,0,0,1
3,2000-01-03,821.0,445.0,0.291521,False,False,21.433333,9.65,85.0,1017.633333,...,0,0,0,0,0,1,0,0,0,1
4,2000-01-03,973.0,1041.0,0.893818,False,False,21.433333,9.65,85.0,1017.633333,...,0,0,0,0,0,1,0,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57469,2024-11-13,630.0,1097.0,1.252763,True,True,23.000000,0.00,50.0,1013.250000,...,0,0,0,1,1,0,0,0,0,1
57470,2024-11-13,630.0,1097.0,1.252763,True,True,23.000000,0.00,50.0,1013.250000,...,0,0,0,1,1,0,0,0,0,1
57471,2024-11-15,1097.0,293.0,0.133531,True,True,23.000000,0.00,50.0,1013.250000,...,0,0,0,1,1,0,0,0,0,1
57472,2024-11-15,1097.0,293.0,0.133531,True,True,23.000000,0.00,50.0,1013.250000,...,0,0,0,1,1,0,0,0,0,1


In [59]:
# проверяем нулевые значение которыемы могли пропустить
df.isnull().sum()

Date                                     0
Winner                                 358
Loser                                  345
W_over_L_rank_difference_normalized      0
W_played_here_before                     0
L_played_here_before                     0
Temperature                              0
Windspeed_10m                            0
Relativehumidity_2m                      0
Surface_pressure                         0
Precipitation                            0
Сloudcover                               0
Shortwave_radiation                      0
W_flag_code                              0
W_weight_kg                              0
W_height_cm                              0
L_flag_code                              0
L_weight_kg                              0
L_height_cm                              0
W_age                                    0
L_age                                    0
Series_ATP250                            0
Series_ATP500                            0
Series_Gran

In [60]:
# Удаление строк с пропущенными значениями в указанных колонках
df = df.dropna(subset=['Winner', 'Loser'], how='any')
df.isnull().sum()

Date                                   0
Winner                                 0
Loser                                  0
W_over_L_rank_difference_normalized    0
W_played_here_before                   0
L_played_here_before                   0
Temperature                            0
Windspeed_10m                          0
Relativehumidity_2m                    0
Surface_pressure                       0
Precipitation                          0
Сloudcover                             0
Shortwave_radiation                    0
W_flag_code                            0
W_weight_kg                            0
W_height_cm                            0
L_flag_code                            0
L_weight_kg                            0
L_height_cm                            0
W_age                                  0
L_age                                  0
Series_ATP250                          0
Series_ATP500                          0
Series_Grand Slam                      0
Series_Internati

In [61]:
# Функция, которая считает h2h_winning_ratio
def calculate_h2h_winning_ratio(data: pd.DataFrame, date: pd.Series, player_1: pd.Series, player_2: pd.Series) -> float:

    past_matches = data[(data['Date'] < date) &
                        (((data['Winner'] == player_1) & (data['Loser'] == player_2)) |
                         ((data['Winner'] == player_2) & (data['Loser'] == player_1)))]

    if len(past_matches) == 0:
        return 0.5

    player_1_wins = (past_matches['Winner'] == player_1).sum()
    player_1_winning_ratio = player_1_wins / len(past_matches)

    return player_1_winning_ratio

In [62]:
# Создание столбцов «player_1» и «player_2» путем случайного подбора игроков
import numpy as np
df['player_1'] = np.where(np.random.rand(len(df)) > 0.5, df['Winner'], df['Loser'])
df['player_2'] = np.where(df['player_1'] == df['Winner'], df['Loser'], df['Winner'])

# Create target variable: 1 if player_1 is the winner, 0 otherwise
df['player_1_win'] = (df['player_1'] == df['Winner']).astype(int)

C:\Users\markz\AppData\Local\Temp\ipykernel_11804\165614756.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['player_1'] = np.where(np.random.rand(len(df)) > 0.5, df['Winner'], df['Loser'])
C:\Users\markz\AppData\Local\Temp\ipykernel_11804\165614756.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['player_2'] = np.where(df['player_1'] == df['Winner'], df['Loser'], df['Winner'])
C:\Users\markz\AppData\Local\Temp\ipykernel_11804\165614756.py:7: SettingWithCopyWarning: 
A value is trying to be set 

In [63]:
# Применяем h2h_winning_ratio
df['player_1_h2h_winning_ratio'] = df.apply(lambda row: calculate_h2h_winning_ratio(df, row['Date'], row['player_1'], row['player_2']), axis=1)

C:\Users\markz\AppData\Local\Temp\ipykernel_11804\3586011511.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['player_1_h2h_winning_ratio'] = df.apply(lambda row: calculate_h2h_winning_ratio(df, row['Date'], row['player_1'], row['player_2']), axis=1)


In [64]:
# Нам теперь не нужны колонки winner и loser
df = df.drop(df[["Winner", "Loser"]], axis = 1)

In [65]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 56772 entries, 0 to 57473
Data columns (total 37 columns):
 #   Column                               Non-Null Count  Dtype         
---  ------                               --------------  -----         
 0   Date                                 56772 non-null  datetime64[ns]
 1   W_over_L_rank_difference_normalized  56772 non-null  float64       
 2   W_played_here_before                 56772 non-null  bool          
 3   L_played_here_before                 56772 non-null  bool          
 4   Temperature                          56772 non-null  float64       
 5   Windspeed_10m                        56772 non-null  float64       
 6   Relativehumidity_2m                  56772 non-null  float64       
 7   Surface_pressure                     56772 non-null  float64       
 8   Precipitation                        56772 non-null  float64       
 9   Сloudcover                           56772 non-null  float64       
 10  Shortwave_radia

In [66]:
# Обработка даты и времени
df.sort_values(by='Date', inplace=True)  # Сортируем по 'Date
df = df.dropna(axis=1)
df

,Date,W_over_L_rank_difference_normalized,W_played_here_before,L_played_here_before,Temperature,Windspeed_10m,Relativehumidity_2m,Surface_pressure,Precipitation,Сloudcover,...,Court_Indoor,Court_Outdoor,Surface_Carpet,Surface_Clay,Surface_Grass,Surface_Hard,player_1,player_2,player_1_win,player_1_h2h_winning_ratio
0,2000-01-03,0.200671,False,False,21.433333,9.650000,85.000000,1017.633333,0.0,72.000000,...,0,1,0,0,0,1,967.0,118.0,0,0.500000
66,2000-01-03,1.214444,False,False,24.016667,17.466667,69.666667,1010.133333,0.0,31.166667,...,0,1,0,0,0,1,79.0,970.0,0,0.500000
65,2000-01-03,-2.212973,False,False,24.016667,17.466667,69.666667,1010.133333,0.0,31.166667,...,0,1,0,0,0,1,79.0,242.0,1,0.500000
64,2000-01-03,1.006805,False,False,24.016667,17.466667,69.666667,1010.133333,0.0,31.166667,...,0,1,0,0,0,1,970.0,1142.0,1,0.500000
63,2000-01-03,2.156733,False,False,24.016667,17.466667,69.666667,1010.133333,0.0,31.166667,...,0,1,0,0,0,1,242.0,438.0,1,0.500000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57469,2024-11-13,1.252763,True,True,23.000000,0.000000,50.000000,1013.250000,0.0,100.000000,...,1,0,0,0,0,1,1097.0,630.0,0,0.400000
57470,2024-11-13,1.252763,True,True,23.000000,0.000000,50.000000,1013.250000,0.0,100.000000,...,1,0,0,0,0,1,1097.0,630.0,0,0.400000
57472,2024-11-15,0.133531,True,True,23.000000,0.000000,50.000000,1013.250000,0.0,100.000000,...,1,0,0,0,0,1,293.0,1097.0,0,0.666667
57471,2024-11-15,0.133531,True,True,23.000000,0.000000,50.000000,1013.250000,0.0,100.000000,...,1,0,0,0,0,1,1097.0,293.0,1,0.333333


In [67]:
# # Извлекаем месяц, день и год из столбца «Date»
df['Month'] = df['Date'].dt.month
df['Day'] = df['Date'].dt.day
df['Year'] = df['Date'].dt.year

In [68]:
# Удаляем оригинальный столбец «Date»
df = df.drop('Date', axis=1)

In [69]:
df.columns

Index(['W_over_L_rank_difference_normalized', 'W_played_here_before',
       'L_played_here_before', 'Temperature', 'Windspeed_10m',
       'Relativehumidity_2m', 'Surface_pressure', 'Precipitation',
       'Сloudcover', 'Shortwave_radiation', 'W_flag_code', 'W_weight_kg',
       'W_height_cm', 'L_flag_code', 'L_weight_kg', 'L_height_cm', 'W_age',
       'L_age', 'Series_ATP250', 'Series_ATP500', 'Series_Grand Slam',
       'Series_International', 'Series_International Gold', 'Series_Masters',
       'Series_Masters 1000', 'Series_Masters Cup', 'Court_Indoor',
       'Court_Outdoor', 'Surface_Carpet', 'Surface_Clay', 'Surface_Grass',
       'Surface_Hard', 'player_1', 'player_2', 'player_1_win',
       'player_1_h2h_winning_ratio', 'Month', 'Day', 'Year'],
      dtype='object')

In [70]:
df

,W_over_L_rank_difference_normalized,W_played_here_before,L_played_here_before,Temperature,Windspeed_10m,Relativehumidity_2m,Surface_pressure,Precipitation,Сloudcover,Shortwave_radiation,...,Surface_Clay,Surface_Grass,Surface_Hard,player_1,player_2,player_1_win,player_1_h2h_winning_ratio,Month,Day,Year
0,0.200671,False,False,21.433333,9.650000,85.000000,1017.633333,0.0,72.000000,0.000000,...,0,0,1,967.0,118.0,0,0.500000,1,3,2000
66,1.214444,False,False,24.016667,17.466667,69.666667,1010.133333,0.0,31.166667,73.166667,...,0,0,1,79.0,970.0,0,0.500000,1,3,2000
65,-2.212973,False,False,24.016667,17.466667,69.666667,1010.133333,0.0,31.166667,73.166667,...,0,0,1,79.0,242.0,1,0.500000,1,3,2000
64,1.006805,False,False,24.016667,17.466667,69.666667,1010.133333,0.0,31.166667,73.166667,...,0,0,1,970.0,1142.0,1,0.500000,1,3,2000
63,2.156733,False,False,24.016667,17.466667,69.666667,1010.133333,0.0,31.166667,73.166667,...,0,0,1,242.0,438.0,1,0.500000,1,3,2000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57469,1.252763,True,True,23.000000,0.000000,50.000000,1013.250000,0.0,100.000000,0.000000,...,0,0,1,1097.0,630.0,0,0.400000,11,13,2024
57470,1.252763,True,True,23.000000,0.000000,50.000000,1013.250000,0.0,100.000000,0.000000,...,0,0,1,1097.0,630.0,0,0.400000,11,13,2024
57472,0.133531,True,True,23.000000,0.000000,50.000000,1013.250000,0.0,100.000000,0.000000,...,0,0,1,293.0,1097.0,0,0.666667,11,15,2024
57471,0.133531,True,True,23.000000,0.000000,50.000000,1013.250000,0.0,100.000000,0.000000,...,0,0,1,1097.0,293.0,1,0.333333,11,15,2024


In [71]:
# Сохраняем финальную версию датасета
df.to_excel('.../datasets/final_dataset.xlsx', index=False)